In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import MetagenomicData.filteringCuratedMetagenomicDataMetadata as fmd
import MetagenomicData.Plotting as pltmg
import MetagenomicData.Rankabundanceplot as ra


In [2]:
metadata_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/MetadataMetgendata.csv"  # Percorso file metadati
metagenomic_data_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/RelabMetgendata.csv"  # Percorso file dati metagenomici
# Caricare i dataset
metadata_csv = pd.read_csv(metadata_file)
metagenomic_data = pd.read_csv(metagenomic_data_file)

C:\Users\spigno\AppData\Local\Temp\ipykernel_28652\1329612460.py:4: DtypeWarning: Columns (21,22,25,28,29,30,32,33,35,38,51,52,53,54,55,56,57,58,63,64,65,67,68,69,74,75,76,80,90,91,92,93,96,104,105,108,109,111,112,116,117,118,119,120,121,126,128,129,134) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_csv = pd.read_csv(metadata_file)


In [18]:
# Initialize the filter
metadata_filter = fmd.MetadataFilter(metadata_csv)

# Apply the filter criteria
filtered_metadata = metadata_filter.filter_data(
    diseases=["healthy"], 
    body_sites=["stool"],
    age_categories=["adult"],
    diets=["omnivore", "vegetarian", "vegan"],
    study_name=["DeFilippisF_2019"]
    )

# Exclude "healthy" manually
#filtered_metadata = filtered_metadata[filtered_metadata["disease"] != "healthy"]
filtered_metadata = filtered_metadata[filtered_metadata["antibiotics_current_use"] != "yes"]

In [19]:
# Obtaining the columns of the metagenomic dataset that correspond to the filtered metadata
selected_columns = filtered_metadata.iloc[:, 0].tolist()  # First column contains the column names

# Filter the metagenomic dataset for the selected columns
df = metagenomic_data[selected_columns]

In [20]:
# Ottieni l'informazione della dieta da aggiungere ai titoli delle colonne
diet_info = filtered_metadata.set_index(filtered_metadata.columns[0])['diet']  # Supponendo che la colonna 'diet' sia in filtered_metadata

# Aggiungi l'informazione della dieta al nome di ciascuna colonna
df.columns = [f"{col}_diet:{diet_info[col]}" for col in df.columns if col in diet_info]


In [23]:
# Definisci un ordine preferito per le categorie di dieta
diet_order = ['vegan', 'vegetarian', 'omnivore']

# Ottieni le informazioni di dieta dalle colonne
column_diet_info = [(col, col.split('_diet:')[1]) for col in df.columns if '_diet:' in col]

# Sposta la dieta all'inizio del nome della colonna
df.columns = [f"{diet}_{col.split('_diet:')[0]}" for col, diet in column_diet_info]

# Aggiorna column_diet_info con i nuovi nomi di colonne
column_diet_info = [(col, col.split('_')[0]) for col in df.columns]

# Ordina le colonne in base all'ordine delle categorie di dieta
sorted_columns = sorted(column_diet_info, key=lambda x: diet_order.index(x[1]) if x[1] in diet_order else len(diet_order))

# Aggiorna le colonne del dataframe con l'ordine desiderato
df = df[[col for col, _ in sorted_columns]]



In [24]:
df.to_csv('metagenomicDeFilippis2.csv', index=True)

# Salva la metadata filtrata in un file CSV
filtered_metadata.to_csv('metadataDeFilippis2.csv', index=False)

In [8]:
# Select only the columns that have at least 100 non-null/non-zero values
df_species = df.loc[:, df.astype(bool).sum(axis=0) >= 100]

# Filter the metadata to keep only the rows that correspond to the columns of df_species
metadata = filtered_metadata[filtered_metadata.iloc[:, 0].isin(df_species.columns)]

# Transpose the dataframe and replace zeros with NaN
df_transposed = df_species.transpose()
df_transposed.replace(0, np.nan, inplace=True)

# Convert the dataframe to a numpy array
data = df_transposed.to_numpy()
print(len(data))

33


In [7]:
import numpy as np
import pandas as pd

# 1. Ottieni le colonne del dataset metagenomico corrispondenti alla metadata filtrata
selected_columns = filtered_metadata.iloc[:, 0].tolist()  # La prima colonna contiene i nomi delle colonne

# 2. Filtra il dataset metagenomico per le colonne selezionate
df = metagenomic_data[selected_columns]

# 3. Seleziona solo le colonne con almeno 100 valori non nulli/non zero
df_species = df.loc[:, df.astype(bool).sum(axis=0) >= 100]

# 4. Filtra la metadata per mantenere solo le righe che corrispondono alle colonne di df_species
metadata = filtered_metadata[filtered_metadata.iloc[:, 0].isin(df_species.columns)]

# 5. Riordina le colonne di df_species secondo l'ordine presente nella metadata filtrata
df_species = df_species[metadata.iloc[:, 0].tolist()]

# 6. Trasponi il dataframe e sostituisci gli zeri con NaN
df_transposed = df_species.transpose()
#df_species.replace(0, np.nan, inplace=True)

# 7. Converti il dataframe in un array numpy
data = df_transposed.to_numpy()

# 8. Verifica la lunghezza dei dati per assicurarti che il numero di righe sia corretto
print("Number of species including NAN (rows):", len(df_species))
print("Number of features (columns):", df_species.shape[1])
# Salva il dataframe trasposto in un file CSV
df.to_csv('metagenomicDeFilippis.csv', index=True)

# Salva la metadata filtrata in un file CSV
filtered_metadata.to_csv('metadataDeFilippis.csv', index=False)



Number of species including NAN (rows): 2055
Number of features (columns): 33
I file CSV sono stati salvati con successo.


In [ ]:
import MetagenomicData.Plotting as pltmg
import MetagenomicData.Rankabundanceplot as ra
plotter = pltmg.HistogramPlotter(data, metadata,column_name="diet", bins=np.logspace(-4, 1, 15),  pdf_filename='SAD_alldiet.pdf') # se non metto bins, usa i valori di default

# Chiamata al metodo per generare e salvare gli istogrammi
plotter.plot_histograms(row=1,col=2)

In [ ]:
plotter = ra.RankAbundancePlotter(data, metadata, column_name="body_site", pdf_filename='RANK_alldiet.pdf')
plotter.plot_histograms()